<a href="https://colab.research.google.com/github/yashnagrawal/image_classification_tf_functional/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Importing Packages

In [ ]:
import tensorflow as tf
import h5py
import numpy as np

# 2) Load dataset

In [ ]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# 3) Preprocess input

In [ ]:
def convert_to_one_hot(Y, C):
  new_Y = np.eye(C)[Y.reshape(-1)]
  return new_Y

In [ ]:
X_train = X_train_orig/255.0
Y_train = convert_to_one_hot(Y_train_orig, len(classes))

X_test = X_test_orig/255.0
Y_test = convert_to_one_hot(Y_test_orig, len(classes))

In [ ]:
print("X train: ", X_train.shape)
print("Y train: ", Y_train.shape)
print("X test: ", X_test.shape)
print("Y test: ", Y_test.shape)

X train:  (1080, 64, 64, 3)
Y train:  (1080, 6)
X test:  (120, 64, 64, 3)
Y test:  (120, 6)


# 4) Define model

In [ ]:
def conv_model():
  """
  CONV2D(8 4 by 4 filters, stride 1, padding is "SAME") -> RELU -> MAXPOOL(8 by 8 filter size and an 8 by 8 stride, padding is "SAME") -> CONV2D(16 2 by 2 filters, stride 1, padding is "SAME") -> RELU -> MAXPOOL(4 by 4 filter size and a 4 by 4 stride, padding is "SAME") -> FLATTEN -> DENSE(6 neurons and a softmax activation)
  Return:
    model -- tensor model
  """
  input_img = tf.keras.Input(shape = (64, 64, 3))
  Z1 = tf.keras.layers.Conv2D(filters = 8, kernel_size = (4, 4), strides = 1, padding = 'SAME')(input_img)
  A1 = tf.keras.layers.ReLU()(Z1)
  P1 = tf.keras.layers.MaxPooling2D(pool_size = (8, 8), strides = (8, 8), padding = 'SAME')(A1)

  Z2 = tf.keras.layers.Conv2D(filters = 16, kernel_size = (2, 2), strides = 1, padding = 'SAME')(P1)
  A2 = tf.keras.layers.ReLU()(Z2)
  P2 = tf.keras.layers.MaxPooling2D(pool_size = (4, 4), strides = (4, 4), padding = 'SAME')(A2)

  F = tf.keras.layers.Flatten()(P2)
  output = tf.keras.layers.Dense(units = 6, activation = 'softmax')(F)

  model = tf.keras.Model(inputs = input_img, outputs = output)
  return model

In [ ]:
sign_model = conv_model()

In [ ]:
sign_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
sign_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 8)         392       
                                                                 
 re_lu (ReLU)                (None, 64, 64, 8)         0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 8, 8, 8)           0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          528       
                                                                 
 re_lu_1 (ReLU)              (None, 8, 8, 16)          0         
                                                             

# 5) Training the model

In [22]:
sign_model.fit(x = X_train, y = Y_train, epochs = 100, validation_data = (X_test, Y_test), batch_size = 64)

Epoch 1/100
17/17 [==============================] - 2s 94ms/step - loss: 0.5026 - accuracy: 0.8500 - val_loss: 0.6245 - val_accuracy: 0.7417
Epoch 2/100
17/17 [==============================] - 2s 99ms/step - loss: 0.5069 - accuracy: 0.8417 - val_loss: 0.5928 - val_accuracy: 0.7500
Epoch 3/100
17/17 [==============================] - 2s 91ms/step - loss: 0.5127 - accuracy: 0.8324 - val_loss: 0.5789 - val_accuracy: 0.7750
Epoch 4/100
17/17 [==============================] - 2s 90ms/step - loss: 0.5057 - accuracy: 0.8306 - val_loss: 0.5747 - val_accuracy: 0.7583
Epoch 5/100
17/17 [==============================] - 3s 190ms/step - loss: 0.4889 - accuracy: 0.8463 - val_loss: 0.5871 - val_accuracy: 0.7750
Epoch 6/100
17/17 [==============================] - 4s 212ms/step - loss: 0.5044 - accuracy: 0.8398 - val_loss: 0.5829 - val_accuracy: 0.7750
Epoch 7/100
17/17 [==============================] - 2s 139ms/step - loss: 0.5032 - accuracy: 0.8343 - val_loss: 0.5844 - val_accuracy: 0.7667
Epo